In [650]:
import pandas as pd
import numpy as np
import math
from datetime import date
import datetime
import time

In [651]:
nugt = pd.read_csv("nugt.csv")
dust = pd.read_csv("dust.csv")

nugt = nugt[['Date','Time', 'Close']]
dust = dust[['Date','Time', 'Close']]

nugt = nugt.shift(-1)
dust = dust.shift(-1)

nugt.columns = ['DateJNUG', 'TimeJNUG', 'CloseJNUG']
dust.columns = ['DateJDST', 'TimeJDST', 'CloseJDST']


nugt = nugt[389:779]
dust = dust[390:780]

# nugt = nugt[389:779]
# dust = dust[390:780]

nugt = nugt.reset_index(drop=True)
dust = dust.reset_index(drop=True)

prices = nugt
prices['DateJDST'] = dust['DateJDST']
prices['TimeJDST'] = dust['TimeJDST']
prices['CloseJDST'] = dust['CloseJDST']


In [660]:
longJNUG = False
longJDST = False
ppJNUG = 0.0
ppJDST = 0.0
prevHighJNUG = 0.0
prevHighJDST = 0.0
slJNUG = 0.0
slJDST = 0.0
portJNUG = 10000
portJDST = 10000

tradesJNUG = {'date': [], 'time':[], 'close':[], 'trade signal':[]}
tradesJDST = {'date': [], 'time':[], 'close':[], 'trade signal':[]}
tradesJNUG = pd.DataFrame(data=tradesJNUG)
tradesJDST = pd.DataFrame(data=tradesJDST)

def sl(row):
    global longJNUG, longJDST, ppJNUG, ppJDST, prevHighJNUG, prevHighJDST, slJNUG, slJDST, tradesJNUG, tradesJDST, portJDST, portJNUG 
    
    if ((longJNUG is False) and (longJDST is False)):
        ppJNUG = row['CloseJNUG']
        ppJDST = row['CloseJDST']
        prevHighJNUG = ppJNUG
        prevHighJDST = ppJDST
        slJNUG = row['CloseJNUG'] * 0.995
        slJDST = row['CloseJDST'] * 0.995
        longJNUG = True
        longJDST = True
        tradeJNUG = {'date': [row['DateJNUG']], 'time':[row['TimeJNUG']], 'close':[row['CloseJNUG']], 'trade signal':["Buy"]}
        tradeJDST = {'date': [row['DateJDST']], 'time':[row['TimeJDST']], 'close':[row['CloseJDST']], 'trade signal':["Buy"]}
        tradeJNUG = pd.DataFrame(tradeJNUG)
        tradeJDST = pd.DataFrame(tradeJDST)
        tradesJNUG = tradesJNUG.append(tradeJNUG)
        tradesJDST = tradesJDST.append(tradeJDST)
        
    else:
        if (row['CloseJNUG'] > prevHighJNUG):
            prevHighJNUG = row['CloseJNUG']
            slJNUG = row['CloseJNUG'] * 0.995
        elif (row['CloseJNUG'] <= slJNUG and longJNUG):
            longJNUG = False
            trade = {'date': [row['DateJNUG']], 'time':[row['TimeJNUG']], 'close':[row['CloseJNUG']], 'trade signal':["Sell"]}
            trade = pd.DataFrame(trade)
            tradesJNUG = tradesJNUG.append(trade)
            portJNUG = portJNUG * (slJNUG/ppJNUG)
        
        if (row['CloseJDST'] > prevHighJDST):
            prevHighJDST = row['CloseJDST']
            slJDST = row['CloseJDST'] * 0.995
        elif (row['CloseJDST'] <= slJDST and longJDST):
            longJDST = False
            trade = {'date': [row['DateJDST']], 'time':[row['TimeJDST']], 'close':[row['CloseJDST']], 'trade signal':["Sell"]}
            trade = pd.DataFrame(trade)
            tradesJDST = tradesJDST.append(trade)
            portJDST = portJDST * (slJDST/ppJDST)

        if (not longJNUG and not longJDST):
            ppJNUG = row['CloseJNUG']
            ppJDST = row['CloseJDST']
            prevHighJNUG = ppJNUG
            prevHighJDST = ppJDST
            slJNUG = row['CloseJNUG'] * 0.995
            slJDST = row['CloseJDST'] * 0.995
            longJNUG = True
            longJDST = True
            tradeJNUG = {'date': [row['DateJNUG']], 'time':[row['TimeJNUG']], 'close':[row['CloseJNUG']], 'trade signal':["Buy"]}
            tradeJDST = {'date': [row['DateJDST']], 'time':[row['TimeJDST']], 'close':[row['CloseJDST']], 'trade signal':["Buy"]}
            tradeJNUG = pd.DataFrame(tradeJNUG)
            tradeJDST = pd.DataFrame(tradeJDST)
            tradesJNUG = tradesJNUG.append(tradeJNUG)
            tradesJDST = tradesJDST.append(tradeJDST)

backtest = prices.apply(lambda row: sl(row),axis=1)

In [661]:
tradesJNUG.columns = ['closeJNUG', 'dateJNUG', 'timeJNUG', 'signalJNUG']
tradesJDST.columns = ['closeJDST', 'dateJDST', 'timeJDST', 'signalJDST']

In [662]:
trades = pd.DataFrame()
trades = tradesJNUG
trades

,closeJNUG,dateJNUG,timeJNUG,signalJNUG
0,15.6389,04/24/2019,9:31 AM,Buy
0,15.5300,04/24/2019,9:32 AM,Sell
0,15.5551,04/24/2019,9:36 AM,Buy
0,15.5400,04/24/2019,9:38 AM,Sell
0,15.4470,04/24/2019,10:21 AM,Buy
0,15.6900,04/24/2019,10:43 AM,Sell
0,15.6900,04/24/2019,10:43 AM,Buy
0,16.1700,04/24/2019,11:18 AM,Sell
0,16.1700,04/24/2019,11:18 AM,Buy
0,16.1016,04/24/2019,11:27 AM,Sell


In [663]:
trades['closeJDST'] = tradesJDST['closeJDST']
trades['dateJDST'] = tradesJDST['dateJDST']
trades['timeJDST'] = tradesJDST['timeJDST']
trades['signalJDST'] = tradesJDST['signalJDST']

In [664]:
trades

,closeJNUG,dateJNUG,timeJNUG,signalJNUG,closeJDST,dateJDST,timeJDST,signalJDST
0,15.6389,04/24/2019,9:31 AM,Buy,22.390,04/24/2019,9:31 AM,Buy
0,15.5300,04/24/2019,9:32 AM,Sell,22.510,04/24/2019,9:36 AM,Sell
0,15.5551,04/24/2019,9:36 AM,Buy,22.510,04/24/2019,9:36 AM,Buy
0,15.5400,04/24/2019,9:38 AM,Sell,22.670,04/24/2019,10:21 AM,Sell
0,15.4470,04/24/2019,10:21 AM,Buy,22.670,04/24/2019,10:21 AM,Buy
0,15.6900,04/24/2019,10:43 AM,Sell,22.470,04/24/2019,10:25 AM,Sell
0,15.6900,04/24/2019,10:43 AM,Buy,22.320,04/24/2019,10:43 AM,Buy
0,16.1700,04/24/2019,11:18 AM,Sell,22.180,04/24/2019,10:52 AM,Sell
0,16.1700,04/24/2019,11:18 AM,Buy,21.645,04/24/2019,11:18 AM,Buy
0,16.1016,04/24/2019,11:27 AM,Sell,21.620,04/24/2019,11:41 AM,Sell


In [665]:
portJNUG

10476.225086162318

In [666]:
portJDST

10221.317976210617

In [659]:
len(trades)

25